# Python для сбора и анализа данных

*Алла Тамбовцева, НИУ ВШЭ*

## Практикум по сбору информации о курсах валют
### Часть 1: выгружаем данные

Загружаем необходимые модули и библиотеки:

In [1]:
import requests
from bs4 import BeautifulSoup

Отправляем запрос к странице с курсами валют от РБК (обновляется, результаты могут отличаться в зависимости от времени подключения), забираем исходный код HTML и превращаем его в объект BeautifulSoup:

In [2]:
page = requests.get("https://cash.rbc.ru/")
soup = BeautifulSoup(page.text)

Несмотря на то, что курсы валют в разных банках представлены в табличном виде, настоящей таблицей с тэгом `<table>` этот фрагмент страницы не является. Но это нестрашно, будем искать информацию по разделам с тэгами `<div>`. Найдем все разделы с подходящим классом и сохраним их в список `divs`, это будут «карточки» с информацией о курсе валют в разных банках:

In [3]:
divs = soup.find_all("div",
                     {"class" : "quote__office__one js-one-office"})

Сначала научимся извлекать информацию на примере одного банка:

In [4]:
d = divs[0]

In [5]:
print(d)

<div class="quote__office__one js-one-office">
<div class="quote__office__cell quote__office__one__star">
<div class="js-select-favorites"></div>
</div>
<div class="quote__office__cell">
<a class="quote__office__one__name" href="/cash/bank/103153.html">АКБ АВАНГАРД ДО Монетчиковский</a>
<div class="quote__office__one__phone">+7 495 730-02-37</div>
</div>
<div class="quote__office__cell quote__office__one__button_full">
</div>
<div class="quote__office__cell quote__office__one__rate quote__mode_list_view">
            82.00
        </div>
<div class="quote__office__cell quote__office__one__cash quote__mode_list_view">
            8200.00
        </div>
<div class="quote__office__cell quote__office__one__sell js-filter-pro quote__mode_list_view_pro">
            81.20
                <div class="quote__office__one__diapason">от </div>
</div>
<div class="quote__office__cell quote__office__one__buy js-filter-pro quote__mode_list_view_pro">
            82.00
                <div class="quot

Плюс автоматической выгрузки данных: в исходном коде страницы хранится больше информации, чем мы видим непосредственно как пользователи. В выгруженных «карточках» есть не только информация о покупке валюты, которая отображается на сайте по умолчанию, но и курс при продаже, а также ближайшая к банку станция метро с расстоянием до него.

Найдем ссылку на страницу банка и заберем из нее две части – саму ссылку (`link_href`) и название банка (`link_text`):

In [6]:
link = d.find("a", {"class" : "quote__office__one__name"})
link_href = link.get("href")
link_text = link.text
print(link_href, link_text)

/cash/bank/103153.html АКБ АВАНГАРД ДО Монетчиковский


Аналогичным образом извлечем телефон банка, курс валют при покупке и при продаже (попутно избавимся от лишних отступов и слов в курсах валют):

In [7]:
phone = d.find("div", {"class" : "quote__office__one__phone"}).text

In [8]:
buy = d.find("div", 
             {"class" : "quote__office__cell quote__office__one__rate quote__mode_list_view"}).text
buy = buy.strip()

sell = d.find("div", 
              {"class" : "quote__office__cell quote__office__one__sell js-filter-pro quote__mode_list_view_pro"}).text
sell = sell.replace("от", "").strip()

In [9]:
print(phone, buy, sell)

+7 495 730-02-37 82.00 81.20


Пока все значения оставим в виде строк (тип `str`), преобразование типов выполним в самом конце, когда соберем всю информацию и сохраним в датафрейм.

Теперь найдем ближайшую станцию метро и расстояние до нее (в первом случае тэг `<span>` не имеет атрибута `class`, это его отличительная особенность, поэтому класс будет пустой):

In [10]:
metro = d.find("span", {"class" : None}).text
distance = d.find("span", 
                  {"class" : "quote__office__metro__distance"}).text
print(metro, distance)

Павелецкая (Кольцевая) 642 м


Отлично! Всю информацию по одному банку выгружать мы научились. Напишем функцию, чтобы применить ее потом в цикле ко всем «карточкам» банков в `divs`. Собственно, для написания функции нам достаточно скопировать блоки кода выше и перечислить в `return` итоговые результаты в виде кортежа:

In [11]:
def get_bank(d):
    link = d.find("a", {"class" : "quote__office__one__name"})
    link_href = link.get("href")
    link_text = link.text
    phone = d.find("div", {"class" : "quote__office__one__phone"}).text
    buy = d.find("div", 
                 {"class" : "quote__office__cell quote__office__one__rate quote__mode_list_view"}).text
    buy = buy.strip()
    sell = d.find("div", 
                  {"class" : "quote__office__cell quote__office__one__sell js-filter-pro quote__mode_list_view_pro"}).text
    sell = sell.replace("от", "").strip()
    
    metro = d.find("span", {"class" : None}).text
    distance = d.find("span", 
                  {"class" : "quote__office__metro__distance"}).text
    return link_href, link_text, phone, buy, sell, metro, distance

Попробуем применить функцию ко всем элементам списка `divs`. Давайте напишем цикл с конструкцией `try-except`, чтобы если что, зафиксировать случаи, когда функция не срабатывает из-за ошибок:

In [12]:
banks_info = []
errors = []

for bank in divs:
    try:
        res = get_bank(bank)
        banks_info.append(res)
    except:
        errors.append(bank)

Проверим, сколько у нас банков и сколько ошибок:

In [13]:
print(len(banks_info), len(errors))

219 30


Список с фрагментами кода из `divs`, на которых функция не сработала, оказался непустым. Если мы изучим его элементы, окажется, что это те банки, у которых нет метро поблизости (станция и расстояние не указаны). Скорректируем нашу функцию на случай, если информация о метро отсутствует – добавим конструкцию `try-except`:

In [24]:
def get_bank(d):
    link = d.find("a", {"class" : "quote__office__one__name"})
    link_href = link.get("href")
    link_text = link.text
    phone = d.find("div", {"class" : "quote__office__one__phone"}).text
    buy = d.find("div", {"class" : "quote__office__cell quote__office__one__rate quote__mode_list_view"}).text
    buy = buy.strip()
    sell = d.find("div", {"class" : "quote__office__cell quote__office__one__sell js-filter-pro quote__mode_list_view_pro"}).text
    sell = sell.replace("от", "").strip()
    try:
        metro = d.find("span", {"class" : None}).text
        distance = d.find("span", 
                  {"class" : "quote__office__metro__distance"}).text
    except:
        metro = ""
        distance= ""
    return link_href, link_text, phone, buy, sell, metro, distance

Повторяем выгрузку:

In [25]:
banks_info = []
errors = []

for bank in divs:
    try:
        res = get_bank(bank)
        banks_info.append(res)
    except:
        errors.append(bank)

In [26]:
print(len(banks_info), len(errors))

249 0


Теперь все отлично! Осталось превратить все в датафрейм `pandas` и преобразовать данные.

### Часть 2: обрабатываем данные

Импортируем библиотеку `pandas`:

In [27]:
import pandas as pd

Получаем датафрейм из списка кортежей:

In [28]:
df = pd.DataFrame(banks_info)

Приписываем названия столбцам:

In [29]:
df.columns = ["link", "name", "phone", 
              "buy", "sell", "metro", "distance"]

Сделаем ссылки на страницы банков абсолютными – доклеим к каждой ссылку на главную страницу. Чтобы избежать циклов, опишем нужное преобразование с помощью анонимной lambda-функции и применим ее ко всем элементам столбца `link` через метод `.apply()`:

In [30]:
df["link"] = df["link"].apply(lambda x: "https://cash.rbc.ru" + x)

Пока все столбцы в датафрейме у нас текстовые – в `pandas` вместо `string` текстовый тип назвается `object`:

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   link      249 non-null    object
 1   name      249 non-null    object
 2   phone     249 non-null    object
 3   buy       249 non-null    object
 4   sell      249 non-null    object
 5   metro     249 non-null    object
 6   distance  249 non-null    object
dtypes: object(7)
memory usage: 13.7+ KB


Скорректируем типы столбцов с курсами валют – переделаем их на `float`:

In [32]:
df["buy"] = df["buy"].astype(float)
df["sell"] = df["sell"].astype(float)
df.head(10)

,link,name,phone,buy,sell,metro,distance
0,https://cash.rbc.ru/cash/bank/103153.html,АКБ АВАНГАРД ДО Монетчиковский,+7 495 730-02-37,82.00,81.2,Павелецкая (Кольцевая),642 м
1,https://cash.rbc.ru/cash/bank/101208.html,КБ ЭНЕРГОТРАНСБАНК,+7 495 627-39-06,81.80,80.7,Китай-город (Калужско-Рижская),385 м
2,https://cash.rbc.ru/cash/bank/103085.html,КБ ЮНИСТРИМ ОО № 183,+7 926 331-08-57,82.60,79.3,,
3,https://cash.rbc.ru/cash/bank/103270.html,Банк Акцепт,+7 495 725-39-65 доб. 451,82.00,80.5,Парк Победы (Арбатско-Покровская),175 м
4,https://cash.rbc.ru/cash/bank/101405.html,ПроБанк ДО Зеленоградский,+7 495 347-47-47 доб. 124,82.00,80.3,,
5,https://cash.rbc.ru/cash/bank/103156.html,Сбербанк ДО № 9038/01775,+7 800 555-55-50,83.65,78.6,Площадь Ильича,780 м
6,https://cash.rbc.ru/cash/bank/103159.html,Сбербанк ДО № 9038/01717,+7 800 555-55-50,83.65,78.6,Ясенево,732 м
7,https://cash.rbc.ru/cash/bank/103094.html,Банк ФК Открытие Филиал Центральный ДО Улица 1...,+7 800 700-37-10,82.10,79.9,Улица 1905 года,555 м
8,https://cash.rbc.ru/cash/bank/102928.html,Банк ФК Открытие Филиал Центральный ДО Мичурин...,+7 800 700-37-10,82.10,79.9,Раменки,278 м
9,https://cash.rbc.ru/cash/bank/103013.html,Банк ФК Открытие Филиал Центральный ДО Новый А...,+7 800 700-37-10,82.10,79.9,Смоленская (Филёвская),801 м


Теперь разберемся с расстоянием до метро. 

In [33]:
df["distance"]

0       642 м
1       385 м
2            
3       175 м
4            
        ...  
244     189 м
245     709 м
246     317 м
247     118 м
248    1.2 км
Name: distance, Length: 249, dtype: object

Где-то расстояния указаны в метрах, где-то в километрах. Давайте напишем функцию, которая определяет по строке единицы измерения и возвращает расстояния до метро в метрах:

In [34]:
def get_dist(x):
    if "км" in x:
        km = float(x.split()[0])
        m = km * 1000
    elif "м" in x:
        m = int(x.split()[0])
    else:
        m = None
    return m

Применяем эту функцию к столбцу `distance` и создаем новый столбец `distance_m`:

In [35]:
df["distance_m"] = df["distance"].apply(get_dist)

Уберем старый столбец с расстоянием в текстовом виде. Удалим его с помощью метода `.drop()` и добавим внутри аргумент `inplace=True`, чтобы изменения сохранились в самом датафрейме.

In [36]:
# inplace = True, чтобы не писать
# df = df.drop(columns = ["distance"])

df.drop(columns = ["distance"], inplace = True)

Осталось решить небольшую проблему: в названиях станций иногда фигурирует название линии в круглых скобках. Это может затруднять поиск строк датафрейму по названию станции. Давайте разделим эти значения на части — разобьем строки по открывающей скобке: 

In [37]:
small = df["metro"].str.split("(", expand = True)
small.head()

,0,1
0,Павелецкая,Кольцевая)
1,Китай-город,Калужско-Рижская)
2,,None
3,Парк Победы,Арбатско-Покровская)
4,,None


Выше мы использовали метод `.split()` из коллекции методов для строк `str` внутри `pandas` и добавили аргумент `expand = True`, чтобы результат разбиения «растянулся» на разные столбцы. Без этого аргумента мы получим один столбец, где в ячейках сохранены списки с частями строки:

In [38]:
df["metro"].str.split("(")

0                 [Павелецкая , Кольцевая)]
1         [Китай-город , Калужско-Рижская)]
2                                        []
3      [Парк Победы , Арбатско-Покровская)]
4                                        []
                       ...                 
244                                [Перово]
245                        [Красные ворота]
246                           [Фрунзенская]
247              [Бульвар Дмитрия Донского]
248                        [Красные ворота]
Name: metro, Length: 249, dtype: object

Присвоим столбцам в полученном маленьком датафрейме названия:

In [39]:
small.columns = ["station", "line"]

Теперь осталось убрать в столбце `line` лишнюю закрывающую скобку. Мы можем сделать это, как и ранее, через lambda-функцию, но здесь нужно учесть один нюанс: в этом столбце есть ячейки с пустым типом `None`. Включим в lambda-функцию условие с `if-else`:

In [40]:
# возвращай строку x с заменой, если не None
# иначе возвращай исходное значение x (None)

small["line"] = small["line"].apply(lambda x: 
                                    x.replace(")", "") if x is not None else x)

In [41]:
small.head()

,station,line
0,Павелецкая,Кольцевая
1,Китай-город,Калужско-Рижская
2,,None
3,Парк Победы,Арбатско-Покровская
4,,None


Осталось склеить наш исходный датафрейм `df` с только что полученным датафреймом `small`. Воспользуемся функцией `concat()`, которая склеивает датафреймы, поданные на вход в виде списка:

In [42]:
# axis = 0: по умолчанию склеивает по строкам,
# второй датафрейм дописывается под первым
# axis = 1: склеивает по столбцам,
# второй датафрейм дописывается справа от первого

final = pd.concat([df, small], axis = 1)

Удалим старый столбец со станцией метро, он нам уже не нужен:

In [43]:
final.drop(columns = ["metro"], inplace = True)

Всё! Посмотрим на датафрейм:

In [44]:
final.head()

,link,name,phone,buy,sell,distance_m,station,line
0,https://cash.rbc.ru/cash/bank/103153.html,АКБ АВАНГАРД ДО Монетчиковский,+7 495 730-02-37,82.0,81.2,642.0,Павелецкая,Кольцевая
1,https://cash.rbc.ru/cash/bank/101208.html,КБ ЭНЕРГОТРАНСБАНК,+7 495 627-39-06,81.8,80.7,385.0,Китай-город,Калужско-Рижская
2,https://cash.rbc.ru/cash/bank/103085.html,КБ ЮНИСТРИМ ОО № 183,+7 926 331-08-57,82.6,79.3,NaN,,None
3,https://cash.rbc.ru/cash/bank/103270.html,Банк Акцепт,+7 495 725-39-65 доб. 451,82.0,80.5,175.0,Парк Победы,Арбатско-Покровская
4,https://cash.rbc.ru/cash/bank/101405.html,ПроБанк ДО Зеленоградский,+7 495 347-47-47 доб. 124,82.0,80.3,NaN,,None


В завершение добавим хоть какое-нибудь взаимодействие с пользователем. Допустим, пользователь вводит с клавиатуры через запятую и пробел станции метро, которые его интересуют:

In [45]:
metro = input()

Каховская, Марксистская


Преобразуем ввод в список `stations`:

In [46]:
stations = metro.split(", ")
stations

['Каховская', 'Марксистская']

Выберем из датафрейма строки, где значения в столбце `station` входят в полученный список. Сделаем это универсальным образом – через метод `.isin()`, который проверяет вхождение значений в ячейках в некоторую последовательность:

In [47]:
final[final["station"].isin(stations)]

,link,name,phone,buy,sell,distance_m,station,line
40,https://cash.rbc.ru/cash/bank/103203.html,АКБ Трансстройбанк ДО Таганская,+7 495 786-37-74,81.90,81.00,62.0,Марксистская,None
72,https://cash.rbc.ru/cash/bank/102856.html,Россельхозбанк ул. Марксистская,+7 495 644-02-38,81.85,77.10,163.0,Марксистская,None
99,https://cash.rbc.ru/cash/bank/103284.html,РОСБАНК ДО Таганский,+7 800 234-44-34,84.70,78.20,695.0,Марксистская,None
123,https://cash.rbc.ru/cash/bank/100523.html,Реалист Банк бывший БайкалИнвестБанк,+7 499 968-94-24,81.68,80.90,484.0,Марксистская,None
154,https://cash.rbc.ru/cash/bank/100946.html,Джей энд Ти Банк ДО Таганский,+7 495 662-45-45 доб. 750,81.70,80.70,450.0,Марксистская,None
189,https://cash.rbc.ru/cash/bank/102972.html,Газпромбанк ДО 099/1069,+7 800 707-70-37,83.84,79.71,163.0,Марксистская,None
204,https://cash.rbc.ru/cash/bank/61525.html,НС Банк ДО На Азовской,+7 800 555-43-24,82.62,79.12,282.0,Каховская,None
212,https://cash.rbc.ru/cash/bank/102906.html,КБ ЛОКО-Банк ДО Таганский,+7 495 739-55-55,82.40,79.60,163.0,Марксистская,None
217,https://cash.rbc.ru/cash/bank/61297.html,НС Банк ул. Добровольческая,+7 800 555-43-24,82.62,79.12,807.0,Марксистская,None


Работает! Стоит отметить, что фильтрация в `pandas` устроена следующим образом: условие в квадратных скобках возвращает последовательность из `True` и `False` (результат проверки условия для каждой ячейки в столбце), а затем из датафрейма отбираются строки, на которых было возвращено значение `True`:

In [48]:
final["station"].isin(stations)

0      False
1      False
2      False
3      False
4      False
       ...  
244    False
245    False
246    False
247    False
248    False
Name: station, Length: 249, dtype: bool

Для примера посмотрим на фильтрацию по числовым значениям. Выберем строки, которые соответствуют банкам рядом с введенными станциями и с расстоянием до метро менее 200 метров:

In [50]:
final[(final["station"].isin(stations)) & (final["distance_m"] < 200)]

,link,name,phone,buy,sell,distance_m,station,line
40,https://cash.rbc.ru/cash/bank/103203.html,АКБ Трансстройбанк ДО Таганская,+7 495 786-37-74,81.90,81.00,62.0,Марксистская,None
72,https://cash.rbc.ru/cash/bank/102856.html,Россельхозбанк ул. Марксистская,+7 495 644-02-38,81.85,77.10,163.0,Марксистская,None
189,https://cash.rbc.ru/cash/bank/102972.html,Газпромбанк ДО 099/1069,+7 800 707-70-37,83.84,79.71,163.0,Марксистская,None
212,https://cash.rbc.ru/cash/bank/102906.html,КБ ЛОКО-Банк ДО Таганский,+7 495 739-55-55,82.40,79.60,163.0,Марксистская,None


Оператор `&` соответствует логическому И. Если бы нам нужно было логическое ИЛИ (хотя бы одно из условий верно), пригодился бы оператор `|`:

In [52]:
# или станции из stations, или любые станции 
# с расстоянием менее 100 м до метро

final[(final["station"].isin(stations)) | (final["distance_m"] < 100)]

,link,name,phone,buy,sell,distance_m,station,line
17,https://cash.rbc.ru/cash/bank/102407.html,АКБ Трансстройбанк ОКВКУ Электрозаводская,+7 495 786-37-73 доб. 566,82.20,80.30,73.0,Электрозаводская,Арбатско-Покровская
22,https://cash.rbc.ru/cash/bank/103126.html,Сбербанк ДО № 9038/01654,+7 800 555-55-50,83.65,78.60,44.0,Деловой центр,Большая кольцевая
29,https://cash.rbc.ru/cash/bank/101865.html,Банк ФК Открытие Филиал Центральный ДО Коньково,+7 800 700-37-10,82.10,79.90,76.0,Коньково,None
39,https://cash.rbc.ru/cash/bank/102939.html,АКБ Трансстройбанк ДО Семеновский,+7 495 786-37-74,82.30,80.50,62.0,Семёновская,None
40,https://cash.rbc.ru/cash/bank/103203.html,АКБ Трансстройбанк ДО Таганская,+7 495 786-37-74,81.90,81.00,62.0,Марксистская,None
70,https://cash.rbc.ru/cash/bank/102331.html,АКБ Трансстройбанк ОО Парковый,+7 495 786-37-73 доб. 542,82.00,80.00,80.0,Первомайская,None
72,https://cash.rbc.ru/cash/bank/102856.html,Россельхозбанк ул. Марксистская,+7 495 644-02-38,81.85,77.10,163.0,Марксистская,None
73,https://cash.rbc.ru/cash/bank/102857.html,Россельхозбанк ул. Митинская,+7 495 644-02-38,83.15,77.10,88.0,Митино,None
89,https://cash.rbc.ru/cash/bank/103182.html,КБ ЮНИСТРИМ ОО № 198,+7 925 586-13-68,83.00,79.00,99.0,Речной вокзал,None
99,https://cash.rbc.ru/cash/bank/103284.html,РОСБАНК ДО Таганский,+7 800 234-44-34,84.70,78.20,695.0,Марксистская,None
